# Plotting forming limit curves from forming test data

This analysis starts with data exported from the GOM-correlate software.

Each experiment has 3 sections and each section has data for the major and minor strain. Each experiment is repeated multiple times with multiple sample geometries.

We plot 3 forming limit curves (FLC) from the data using different stopping criteria:
* Formation of a crack in the sample
* First derivative necking 
* Second derivative necking.

Formation of a crack in the material is determined by inspection of the sample images.

First derivative necking determines the necking time to be when the strain in the sample becomes localised. This is determined numerically as the point at which the major strain outside the necking region begins to plateau. 

Second derivative necking considers the second derivative of the major strain to determine the necking time. The point at which the second derivative devaites from a constant value is consdiered the necking point.

In [ ]:
from pathlib import Path
import math
import pprint
import glob
from typing import List, Union
from datetime import datetime

from scipy.signal import find_peaks
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pandas as pd

The fracture time of each sample is determined by visual inspection of the sample images at each time step. The fracture times are listed here, one for each sample.

In [ ]:
fracture_times = \
{'Surfalex_10mm_001': 145,
 'Surfalex_10mm_002': 137,
 'Surfalex_10mm_003': 143,
 'Surfalex_20mm_001': 139,
 'Surfalex_20mm_002': 134,
 'Surfalex_20mm_003': 140,
 'Surfalex_40mm_001': 151,
 'Surfalex_40mm_002': 151,
 'Surfalex_40mm_003': 149,
 'Surfalex_60mm_001': 155,
 'Surfalex_60mm_002': 153,
 'Surfalex_60mm_003': 155,
 'Surfalex_120mm_001': 150,
 'Surfalex_120mm_002': 149,
 'Surfalex_120mm_003': 148,
 'Surfalex_full circle_001': 152,
 'Surfalex_fullcircle_002': 157,
 'Surfalex_fullcircle_003': 151
}

First we load the DIC data from the files up the the time of fracture for each sample.

In [ ]:
def get_time_from_file(path: Path) -> datetime:
    sample_time_string = pd.read_csv(path, skiprows=2, delimiter=";", nrows=1)["date"].values[0]
    return datetime.fromisoformat(sample_time_string)


def load_data(file_directory: str, fracture_time: int) -> Union[List[np.ndarray], float]:
    frames = []
    sample_times = []
    file_list = glob.glob(f"{file_directory}/*.csv")

    # We ignore the first frame since it is before the experiment begins and we ignore frames after fracture
    start_time = get_time_from_file(Path(file_list[1]))

    for index, file_name in enumerate(file_list):
        # Add one to index becuase python is zero based but file numbering is one based.
        if 1 < index + 1 < fracture_time:
            file_path = Path(file_name)
            sample_times.append((get_time_from_file(file_path) - start_time).total_seconds())
            data = np.genfromtxt(file_path, skip_header=6, delimiter=";", usecols=[5])
            frames.append(data)
    return frames, sample_times

major_strain_data = {}
minor_strain_data = {}
frame_times = {}
for experiment_name, fracture_time in tqdm(fracture_times.items()):
    file_directory = f"../Data/Strain data_All stages/{experiment_name}/Section one/major strain"
    major_strain_data[experiment_name], time = load_data(file_directory, fracture_time)    
    frame_times[experiment_name] = time
    
    file_directory = f"../Data/Strain data_All stages/{experiment_name}/Section one/minor strain"
    minor_strain_data[experiment_name], _ = load_data(file_directory, fracture_time)

## 1). FLC from crack formation

In [ ]:
for experiment_name, fracture_time in fracture_times.items():
    # Find the position of the maximum strain as it breaks. 
    max_strain_index = np.argmax(major_strain_data[experiment_name][-1])

    # Collect major strain over time at the fracture position
    major_strain = [timestep[max_strain_index] for timestep in major_strain_data[experiment_name]]

    # Collect the same data but for minor strains
    minor_strain = [timestep[max_strain_index] for timestep in minor_strain_data[experiment_name]]

    plt.plot(minor_strain[:fracture_time - 1], major_strain[:fracture_time - 1], "-x")
plt.xlabel("minor strain")
plt.ylabel("major strain")
plt.title(f"Strain paths for Surflex to crack formation")
plt.savefig("../Results/strain_path_to_break.png", dpi=200)

## 2). FLC from first derivative necking

First derivative necking determines the necking time to be when the strain in the sample becomes localised. This is determined numerically as the point at which the major strain outside the necking region begins to plateau.

In order to measure this we need to collect the evolution of the strain with time at the necking point and away from the necking point. We identify the necking point as the point of maximum strain at the timestep before the sample fails. At the point of sample fracture the disribution of strain is bimodal. We identify the point to measure the strain outside the necking region to be the point equidistant between the two strain peaks.

The plots below show the peaks in strain marked with grey lines and the median point marked by a red line.

In [ ]:
median_strain_index  = {}
for experiment_name, fracture_time in fracture_times.items():
    # Identify peaks in strain at fracture time
    peaks, _ = find_peaks(major_strain_data[experiment_name][-1], distance=50)
    median_strain_index[experiment_name] = int(np.median(peaks))
    
    # Plot the identified peaks to be sure there are two.
    plt.plot(major_strain_data[experiment_name][-1])
    plt.vlines(peaks, plt.ylim()[0], plt.ylim()[1], alpha=0.5, color="k")
    plt.vlines(median_strain_index[experiment_name], plt.ylim()[0], plt.ylim()[1], color="r", alpha=0.5)
    plt.title(f"Strain map of sample: {experiment_name} at fracture time")
    plt.xlabel("Vertical sample position")
    plt.ylabel("Strain")
    plt.show()

We now collect the major and minor strain at the neck and away from the neck. 

We cut off the data from before 20 seconds since there are some early fluctuations which do not represent necking but make identification of the necking harder and the necking does not occur before 20 seconds in any of the samples.

In [ ]:
for experiment_name in fracture_times.keys():
    # Find the index of the data point 20 seconds into the experiment
    initial_index = np.argmin(np.abs(np.array(frame_times[experiment_name]) - 20))
    # Find the position of the maximum strain as it breaks. 
    # We subtract 2, one to get the frame before fracture and one because the data is zero indexed and the files are 1 indexed.
    max_strain_index = np.argmax(major_strain_data[experiment_name][-1])

    # Collect strain over time at the fracture position
    major_strain_at_neck = [timestep[max_strain_index] for timestep in major_strain_data[experiment_name]]
    minor_strain_at_neck = [timestep[max_strain_index] for timestep in minor_strain_data[experiment_name]]

    # Collect strain over time away from the fracture position
    major_strain_away_neck = [timestep[median_strain_index[experiment_name]] for timestep in major_strain_data[experiment_name]]
    minor_strain_away_neck = [timestep[median_strain_index[experiment_name]] for timestep in minor_strain_data[experiment_name]]

    #plt.plot(frame_times[experiment_name][initial_index:], major_strain_at_neck[initial_index:], "x-", label="At neck")
    plt.plot(frame_times[experiment_name][initial_index:], major_strain_away_neck[initial_index:], "x-", label="Away from neck")
    plt.xlabel("Time (s)")
    plt.title(f"Major strain for experiment: {experiment_name} at two points")
    plt.ylabel("Strain")
    plt.legend()
    plt.show()
    

## 3). FLC from second derivative necking

In [ ]:
from ipywidgets import interactive, fixed, Checkbox

def plot_major_strain(experiment_name: str, plot_strain: bool, plot_first_derivative: bool, plot_second_derivative: bool):
    # Find the position of the maximum strain as it breaks. 
    fracture_time = fracture_times[experiment_name]
    max_strain_index = np.argmax(major_strain_data[experiment_name][-1])

    # Collect major strain over time at the fracture position
    major_strain = [timestep[max_strain_index] for timestep in major_strain_data[experiment_name]]

    if plot_strain:
        plt.plot(frame_times[experiment_name][:fracture_time], major_strain[:fracture_time], "x-")
    if plot_first_derivative or plot_second_derivative:
        first_derivative = np.gradient(major_strain, frame_times[experiment_name])
    if plot_first_derivative:
        plt.plot(frame_times[experiment_name][:fracture_time], first_derivative[:fracture_time], "x-")
    if plot_second_derivative:
        second_derivative = np.gradient(first_derivative, frame_times[experiment_name])
        plt.plot(frame_times[experiment_name][:fracture_time], second_derivative[:fracture_time], "x-")
    plt.xlabel("Time (s)")
    plt.ylabel("major strain")
    plt.title(f"{experiment_name}")
    plt.show()
    
print()
    
interactive(plot_major_strain,
            experiment_name=list(fracture_times.keys()),
            plot_strain=Checkbox(),
            plot_first_derivative=Checkbox(),
            plot_second_derivative=Checkbox())

In [ ]:
for experiment_name, fracture_time in fracture_times.items():
    # Find the position of the maximum strain as it breaks. 
    max_strain_index = np.argmax(major_strain_data[experiment_name][-1])

    # Collect major strain over time at the fracture position
    major_strain = [timestep[max_strain_index] for timestep in major_strain_data[experiment_name]]
    
    # Collect the same data but for minor strains
    minor_strain = [timestep[max_strain_index] for timestep in minor_strain_data[experiment_name]]

    # fit a polynomial

    # get the second derivative
    first_derivative = np.gradient(fitted_strain, frame_times[experiment_name])
    second_derivative = np.gradient(first_derivative, frame_times[experiment_name])

    # Find the index where the second derivative reaches 10% of its maximum value.
    cut_criterion = 0.1 * np.max(second_derviative)
    cut_index = np.argmin(np.abs(second_derviative - cut_criterion))
    
    plt.plot(minor_strain[:cut_index], major_strain[:cut_index], "-x")
plt.xlabel("minor strain")
plt.ylabel("major strain")
plt.title(f"Strain paths for Surflex to second derivative necking")
plt.savefig("../Results/strain_path_to_second_derivative_necking.png", dpi=200)